<a href="https://colab.research.google.com/github/carlosmatherson/SULI-Project/blob/main/Smiles2Graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Packages


In [ ]:
#!pip install spektral
#!pip install rdkit

In [ ]:
# general tools
import numpy as np
import pandas as pd

# Spektral
import spektral as sp
from spektral.data.graph import Graph
from spektral.data import Dataset
from spektral.data import BatchLoader
from spektral.data import SingleLoader
from spektral.data import MixedLoader
from spektral.data import DisjointLoader
from spektral.data import Loader
from spektral.data import PackedBatchLoader
from spektral.models import GeneralGNN
from spektral.models import GCN
from spektral.layers import GCNConv, ECCConv, GlobalSumPool, GraphMasking

# RDkit
from rdkit import Chem
from rdkit.Chem.rdmolops import GetAdjacencyMatrix

# TensorFlow & Keras
import tensorflow as tf

from keras.layers import Dense
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import RootMeanSquaredError

# Featurization Functions

Onehot Encoding: Maps input elements x which are not in the permitted list to the last element of the permitted list.

In [ ]:
# Onehot Encoding
def one_hot_encoding(x, permitted_list):

    if x not in permitted_list:
        x = permitted_list[-1]

    binary_encoding = [int(boolean_value) for boolean_value in list(map(lambda s: x == s, permitted_list))]

    return binary_encoding

Atom Featurization: Takes an RDKit atom object as input and gives a 1d-numpy array of atom features as output.

In [ ]:
# Atom Featurization
def get_atom_features(atom, 
                      use_chirality = True, 
                      hydrogens_implicit = True):

    # define list of permitted atoms
    
    permitted_list_of_atoms =  ['C','N','O','S','F','Si','P','Cl','Br','Mg','Na','Ca','Fe','As','Al','I', 'B','V','K','Tl','Yb','Sb','Sn','Ag','Pd','Co','Se','Ti','Zn', 'Li','Ge','Cu','Au','Ni','Cd','In','Mn','Zr','Cr','Pt','Hg','Pb','Unknown']
    
    if hydrogens_implicit == False:
        permitted_list_of_atoms = ['H'] + permitted_list_of_atoms
    
    # compute atom features
    
    atom_type_enc = one_hot_encoding(str(atom.GetSymbol()), permitted_list_of_atoms)
    
    n_heavy_neighbors_enc = one_hot_encoding(int(atom.GetDegree()), [0, 1, 2, 3, 4, "MoreThanFour"])
    
    formal_charge_enc = one_hot_encoding(int(atom.GetFormalCharge()), [-3, -2, -1, 0, 1, 2, 3, "Extreme"])
    
    hybridisation_type_enc = one_hot_encoding(str(atom.GetHybridization()), ["S", "SP", "SP2", "SP3", "SP3D", "SP3D2", "OTHER"])
    
    is_in_a_ring_enc = [int(atom.IsInRing())]
    
    is_aromatic_enc = [int(atom.GetIsAromatic())]
    
    atomic_mass_scaled = [float((atom.GetMass() - 10.812)/116.092)]
    
    vdw_radius_scaled = [float((Chem.GetPeriodicTable().GetRvdw(atom.GetAtomicNum()) - 1.5)/0.6)]
    
    covalent_radius_scaled = [float((Chem.GetPeriodicTable().GetRcovalent(atom.GetAtomicNum()) - 0.64)/0.76)]

    atom_feature_vector = atom_type_enc + n_heavy_neighbors_enc + formal_charge_enc + hybridisation_type_enc + is_in_a_ring_enc + is_aromatic_enc + atomic_mass_scaled + vdw_radius_scaled + covalent_radius_scaled
                                    
    if use_chirality == True:
        chirality_type_enc = one_hot_encoding(str(atom.GetChiralTag()), ["CHI_UNSPECIFIED", "CHI_TETRAHEDRAL_CW", "CHI_TETRAHEDRAL_CCW", "CHI_OTHER"])
        atom_feature_vector += chirality_type_enc
    
    if hydrogens_implicit == True:
        n_hydrogens_enc = one_hot_encoding(int(atom.GetTotalNumHs()), [0, 1, 2, 3, 4, "MoreThanFour"])
        atom_feature_vector += n_hydrogens_enc

    return np.array(atom_feature_vector)

Bond Featurization: Takes an RDKit bond object as input and gives a 1d-numpy array of bond features as output.

In [ ]:
# Bond Featurisation
def get_bond_features(bond, 
                      use_stereochemistry = True):

    permitted_list_of_bond_types = [Chem.rdchem.BondType.SINGLE, Chem.rdchem.BondType.DOUBLE, Chem.rdchem.BondType.TRIPLE, Chem.rdchem.BondType.AROMATIC]

    bond_type_enc = one_hot_encoding(bond.GetBondType(), permitted_list_of_bond_types)
    
    bond_is_conj_enc = [int(bond.GetIsConjugated())]
    
    bond_is_in_ring_enc = [int(bond.IsInRing())]
    
    bond_feature_vector = bond_type_enc + bond_is_conj_enc + bond_is_in_ring_enc
    
    if use_stereochemistry == True:
        stereo_type_enc = one_hot_encoding(str(bond.GetStereo()), ["STEREOZ", "STEREOE", "STEREOANY", "STEREONONE"])
        bond_feature_vector += stereo_type_enc

    return np.array(bond_feature_vector)

Dablander, Markus. How to Turn a SMILES String into a Molecular Graph for Pytorch Geometric | Oxford Protein Informatics Group. https://www.blopig.com/blog/2022/02/how-to-turn-a-smiles-string-into-a-molecular-graph-for-pytorch-geometric/. Accessed 11 July 2022.


# Dataset Creation Function

In [ ]:
class MyDataset(Dataset):
  def read(self):
    polydata = pd.read_csv("https://raw.githubusercontent.com/carlosmatherson/SULI-Project/main/SMILES_Density_Tg_Mt.csv")
    graph_list = makeGraphObjList(polydata["SMILES"], polydata["Tg"])
    return graph_list

makeGraphObjList: Creates Spektral graph dataset from smiles and labels.

Inputs: 
*   x_smiles = [smiles_1, smiles_2, ....] ... a list of SMILES strings
*   y = [y_1, y_2, ...] ... a list of numerial labels for the SMILES strings

Outputs:

*   data_list = [G_1, G_2, ...] ... a list of spektral.data.graph.Graph objects *that* can readily be used for machine learning






In [ ]:
def makeGraphObjList(x_smiles, y):
    
    graph_list = []

    for (smiles, y_val) in zip(x_smiles, y):
        
        # convert SMILES to RDKit mol object
        mol = Chem.MolFromSmiles(smiles)
        
        # get feature dimensions
        n_nodes = mol.GetNumAtoms()
        n_edges = 2*mol.GetNumBonds()
        unrelated_smiles = "O=O"
        unrelated_mol = Chem.MolFromSmiles(unrelated_smiles)
        n_node_features = len(get_atom_features(unrelated_mol.GetAtomWithIdx(0)))
        n_edge_features = len(get_bond_features(unrelated_mol.GetBondBetweenAtoms(0,1)))

        # construct adjacency matrix (n_nodes, n_nodes), full matrix
        A = np.array(GetAdjacencyMatrix(mol)) # full matrix

        # construct edge index array Ei of shape (n_edges, 2), sparse matrix
        Ei = np.transpose(np.nonzero(A)) # vertical format
        (rows, cols) = np.transpose(Ei)
        #Ei = np.stack([rows, cols])

        # construct node feature matrix Xn of shape (n_nodes, n_node_features)
        X = np.zeros((n_nodes, n_node_features))
        for atom in mol.GetAtoms():
            X[atom.GetIdx(), :] = get_atom_features(atom)
        
        # construct edge feature array Ef of shape (n_edges, n_edge_features)
        E = np.zeros((n_edges, n_edge_features))
        for (k, (i,j)) in enumerate(zip(rows, cols)):
            E[k] = get_bond_features(mol.GetBondBetweenAtoms(int(i),int(j)))
        sp.utils.sparse.reorder(Ei, E)
        #E = np.hstack([np.transpose(rows),np.transpose(cols),np.transpose(E[-1])])
        # want to try and make E.shape = (n_nodes,n_nodes,n_edge_features)
        
        # construct label tensor
        Y = np.array([y_val])

        # construct Pytorch Geometric data object and append to data list

        #print(Ei.shape,E.shape)
        graph_list.append(Graph(x=X, a=A, e=E, y=Y))
        
    return graph_list 

# Model

In [ ]:
#load Data
data = MyDataset()

In [ ]:
# parameters
F = data.n_node_features  # Dimension of node features, 79
S = data.n_edge_features  # Dimension of edge features, 10
n_out = data.n_labels  # Dimension of the target

# config
learning_rate = 1e-3  # Learning rate
epochs = 50  # Number of training epochs
batch_size = 15  # Batch size

In [ ]:
# train/test split
idxs = np.random.permutation(len(data))
split = int(0.7 * len(data))
idx_tr, idx_te = np.split(idxs, [split])
dataset_tr, dataset_te = data[idx_tr], data[idx_te]

In [ ]:
#model
class Net(Model):
    def __init__(self):
        super().__init__()
        #self.masking = GraphMasking()
        self.conv1 = GCNConv(100, activation="relu")
        self.conv2 = GCNConv(100, activation="relu")
        self.conv3 = GCNConv(100, activation="relu")
        self.conv4 = GCNConv(100, activation="relu")
        self.conv5 = GCNConv(100, activation="relu")
        self.conv6 = GCNConv(100, activation="relu")
        self.global_pool = GlobalSumPool()
        self.dense = Dense(n_out)

    def call(self, inputs):
        x, a = inputs[0], inputs[1]
        #x = self.masking(x)
        x = self.conv1([x, a])
        x = self.conv2([x, a])
        output = self.global_pool(x)
        output = self.dense(output)


        return output


model = Net()
optimizer = Adam(learning_rate)
model.compile(optimizer=optimizer, metrics=[RootMeanSquaredError()], loss="mean_absolute_percentage_error") #loss="mse")

In [ ]:
loader_tr = BatchLoader(dataset_tr, batch_size=batch_size, mask=True)
model.fit(loader_tr.load(), steps_per_epoch=loader_tr.steps_per_epoch, epochs=epochs)

Epoch 1/50


/usr/local/lib/python3.7/dist-packages/spektral/data/utils.py:221: UserWarning: you are shuffling a 'MyDataset' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  np.random.shuffle(a)
/usr/local/lib/python3.7/dist-packages/spektral/layers/convolutional/conv.py:94: UserWarning: The adjacency matrix of dtype <dtype: 'int32'> is incompatible with the dtype of the node features <dtype: 'float32'> and has been automatically cast to <dtype: 'float32'>.
  f"The adjacency matrix of dtype {a.dtype} is incompatible with the dtype "
/usr/local/lib/python3.7/dist-packages/tensorflow/python/autograph/impl/api.py:371: UserWarning: The adjacency matrix of dtype <dtype: 'int32'> is incompatible with the dtype of the node features <dtype: 'float32'> and has been automatically cast to <dtype: 'float32'>.
  return py_builtins.overload_of(f)(*args)


21/21 [==============================] - 1s 21ms/step - loss: 110241288.0000 - root_mean_squared_error: 99.5358
Epoch 2/50
21/21 [==============================] - 0s 23ms/step - loss: 144489104.0000 - root_mean_squared_error: 120.5985
Epoch 3/50
21/21 [==============================] - 0s 21ms/step - loss: 96518648.0000 - root_mean_squared_error: 102.7054
Epoch 4/50
21/21 [==============================] - 0s 17ms/step - loss: 56925820.0000 - root_mean_squared_error: 113.5638
Epoch 5/50
21/21 [==============================] - 0s 21ms/step - loss: 83087872.0000 - root_mean_squared_error: 106.2787
Epoch 6/50
21/21 [==============================] - 0s 18ms/step - loss: 31708658.0000 - root_mean_squared_error: 112.0573
Epoch 7/50
21/21 [==============================] - 0s 18ms/step - loss: 26363172.0000 - root_mean_squared_error: 113.9080
Epoch 8/50
21/21 [==============================] - 0s 19ms/step - loss: 6435811.0000 - root_mean_squared_error: 106.9020
Epoch 9/50
21/21 [=========

In [ ]:
print("Testing model")
loader_te = BatchLoader(dataset_te, batch_size=batch_size, mask=True)
loss = model.evaluate(loader_te.load(), steps=loader_te.steps_per_epoch)
print("Done. Test loss: {}".format(loss))

Testing model


/usr/local/lib/python3.7/dist-packages/spektral/data/utils.py:221: UserWarning: you are shuffling a 'MyDataset' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  np.random.shuffle(a)
/usr/local/lib/python3.7/dist-packages/tensorflow/python/autograph/impl/api.py:371: UserWarning: The adjacency matrix of dtype <dtype: 'int32'> is incompatible with the dtype of the node features <dtype: 'float32'> and has been automatically cast to <dtype: 'float32'>.
  return py_builtins.overload_of(f)(*args)


9/9 [==============================] - 0s 15ms/step - loss: 98.4017 - root_mean_squared_error: 113.9347
Done. Test loss: [98.40169525146484, 113.93470764160156]
